In [ ]:
import requests
import zipfile
import pandas as pd
import numpy as np
zip_arch_local_path = '/tmp/hw_archive.zip'

#### get zip archive

In [ ]:
resp = requests.get(r'https://u.netology.ngcdn.ru/backend/uploads/lms/attachments/files/data/15259/%D0%B4%D0%BE%D0%BF_%D0%BC%D0%B0%D1%82%D0%B5%D1%80%D0%B8%D0%B0%D0%BB%D1%8B_%D0%BA_%D0%BF%D0%B0%D0%BD%D0%B4%D0%B0%D1%81_1_%D0%B8_2.zip')
with open(zip_arch_local_path, 'wb') as arch_file:
    arch_file.write(resp.content)
    arch_file.flush()

# Задание 1

Для датафрейма log из материалов занятия создайте столбец source_type по следующим правилам:

* если источник traffic_source равен yandex или google, то в source_type ставится organic
* для источников paid и email из России - ставим ad
* для источников paid и email не из России - ставим other
* все остальные варианты берем из traffic_source без изменений



#### get `log_df`

In [ ]:
with zipfile.ZipFile(zip_arch_local_path, mode='r', compression=zipfile.ZIP_DEFLATED) as arch:
    with arch.open(r'visit_log.csv', 'r') as visits:
        log_df = pd.read_csv(visits, header=0, sep=';')

In [ ]:
log_df['region'] = log_df['region'].str.lower()
log_df['traffic_source'] = log_df['traffic_source'].str.lower()
log_df.head(9)

,timestamp,visit_id,url,region,user_id,traffic_source
0,1549980692,e3b0c44298,https://host.ru/3c19b4ef7371864fa3,russia,b1613cc09f,yandex
1,1549980704,6e340b9cff,https://host.ru/c8d9213a31839f9a3a,russia,4c3ec14bee,direct
2,1549980715,96a296d224,https://host.ru/b8b58337d272ee7b15,russia,a8c40697fb,yandex
3,1549980725,709e80c884,https://host.ru/b8b58337d272ee7b15,russia,521ac1d6a0,yandex
4,1549980736,df3f619804,https://host.ru/b8b58337d272ee7b15,russia,d7323c571c,yandex
5,1549980742,8855508aad,https://host.ru/df646c3676cc259fa0,russia,fc43898e47,yandex
6,1549980742,b0f66adc83,https://host.ru/b8b58337d272ee7b15,russia,13fc55e781,paid
7,1549980754,837885c8f8,https://host.ru/108ce4b365afb7b88e,russia,cb5082b6f6,direct
8,1549980760,af5570f5a1,https://host.ru/3004a8273caeef2867,china,45664f7af2,direct


#### solution with 'apply'

In [ ]:
def get_source_type(row):
    if (row['traffic_source'] in {'yandex', 'google'}):
        return 'organic'
    if (row['traffic_source'] in {'paid', 'email'}):
        if (row['region'] == 'russia'):
            return 'ad'
        else:
            return 'other'
    return row['traffic_source']

In [ ]:
%%time

log_df['source_type'] = log_df.apply(get_source_type, axis=1)

CPU times: user 262 ms, sys: 5.56 ms, total: 267 ms
Wall time: 270 ms


In [ ]:
log_df.head(9)

,timestamp,visit_id,url,region,user_id,traffic_source,source_type
0,1549980692,e3b0c44298,https://host.ru/3c19b4ef7371864fa3,russia,b1613cc09f,yandex,organic
1,1549980704,6e340b9cff,https://host.ru/c8d9213a31839f9a3a,russia,4c3ec14bee,direct,direct
2,1549980715,96a296d224,https://host.ru/b8b58337d272ee7b15,russia,a8c40697fb,yandex,organic
3,1549980725,709e80c884,https://host.ru/b8b58337d272ee7b15,russia,521ac1d6a0,yandex,organic
4,1549980736,df3f619804,https://host.ru/b8b58337d272ee7b15,russia,d7323c571c,yandex,organic
5,1549980742,8855508aad,https://host.ru/df646c3676cc259fa0,russia,fc43898e47,yandex,organic
6,1549980742,b0f66adc83,https://host.ru/b8b58337d272ee7b15,russia,13fc55e781,paid,ad
7,1549980754,837885c8f8,https://host.ru/108ce4b365afb7b88e,russia,cb5082b6f6,direct,direct
8,1549980760,af5570f5a1,https://host.ru/3004a8273caeef2867,china,45664f7af2,direct,direct


#### solution with masks

In [ ]:
%%time

log_df.loc[log_df['traffic_source'].isin({'yandex', 'google'}), 'source_type_2'] = 'organic'
paid_email_mask = log_df['traffic_source'].isin({'paid', 'email'})
log_df.loc[paid_email_mask & (log_df['region'] == 'russia'), 'source_type_2'] = 'ad'
log_df.loc[paid_email_mask & (log_df['region'] != 'russia'), 'source_type_2'] = 'other'
log_df.loc[log_df['source_type_2'].isna(), 'source_type_2'] = log_df['traffic_source'].loc[log_df['source_type_2'].isna()]

CPU times: user 23.8 ms, sys: 1.97 ms, total: 25.8 ms
Wall time: 25.1 ms


In [ ]:
log_df.loc[log_df['source_type'] != log_df['source_type_2']].head(9)

,timestamp,visit_id,url,region,user_id,traffic_source,source_type,source_type_2


# Задание 2

В файле URLs.txt содержатся url страниц новостного сайта. Вам необходимо отфильтровать его по адресам страниц с текстами новостей. Известно, что шаблон страницы новостей имеет внутри url следующую конструкцию: /, затем 8 цифр, затем дефис. Выполните следующие действия:

Прочитайте содержимое файла с датафрейм

Отфильтруйте страницы с текстом новостей, используя метод str.contains и регулярное выражение в соответствии с заданным шаблоном



#### get `url_df`

In [ ]:
with zipfile.ZipFile(zip_arch_local_path, mode='r', compression=zipfile.ZIP_DEFLATED) as arch:
    with arch.open('URLs.txt', mode='r') as urls:
        url_df = pd.read_csv(filepath_or_buffer=urls, header=0)

In [ ]:
url_df.head(9)

,url
0,/world/
1,/latest/
2,/?updated=top
3,/politics/36188461-s-marta-zhizn-rossiyan-susc...
4,/world/36007585-tramp-pridumal-kak-reshit-ukra...
5,/science/36157853-nasa-sobiraet-ekstrennuyu-pr...
6,/video/36001498-poyavilis-pervye-podrobnosti-g...
7,/world/36007585-tramp-pridumal-kak-reshit-ukra...
8,/science/


#### solution

In [ ]:
url_df.loc[url_df['url'].str.contains('/\d{8}-', regex=True)]

,url
3,/politics/36188461-s-marta-zhizn-rossiyan-susc...
4,/world/36007585-tramp-pridumal-kak-reshit-ukra...
5,/science/36157853-nasa-sobiraet-ekstrennuyu-pr...
6,/video/36001498-poyavilis-pervye-podrobnosti-g...
7,/world/36007585-tramp-pridumal-kak-reshit-ukra...
...,...
88,/cis/35984145-kreml-prokommentiroval-soobschen...
89,/video/36071019-olimpiyskie-obekty-rio-prevrat...
90,/science/36151301-nazvano-posledstvie-zloupotr...
91,/incidents/36027330-vospitatelnitsu-zatravili-...


# Задание 3

Используйте файл с оценками фильмов ml-latest-small/ratings.csv. Посчитайте среднее время жизни пользователей, которые выставили более 100 оценок. Под временем жизни понимается разница между максимальным и минимальным значением столбца timestamp для данного значения userId.

#### get `ratings_df`

In [ ]:
with zipfile.ZipFile(zip_arch_local_path, mode='r', compression=zipfile.ZIP_DEFLATED) as arch:
    with arch.open('ml-latest-small/ratings.csv', mode='r') as ratings:
        ratings_df = pd.read_csv(filepath_or_buffer=ratings, header=0, sep=',', usecols=['userId', 'timestamp'], dtype={'userId': np.int64, 'timestamp': np.int64})
        ratings_df['timestamp'] = pd.to_datetime(ratings_df['timestamp'], unit='s', origin='unix')

In [ ]:
ratings_df.head(9)

,userId,timestamp
0,1,2009-12-14 02:52:24
1,1,2009-12-14 02:52:59
2,1,2009-12-14 02:53:02
3,1,2009-12-14 02:53:05
4,1,2009-12-14 02:53:25
5,1,2009-12-14 02:52:31
6,1,2009-12-14 02:53:07
7,1,2009-12-14 02:52:28
8,1,2009-12-14 02:52:05


#### solution

In [ ]:
ratings_df.groupby(['userId'])\
    .agg(cnt=('timestamp', 'count'), min_ts=('timestamp', 'min'), max_ts=('timestamp', 'max'))\
    .query('cnt > 100')\
    .assign(delta=lambda row: row['max_ts'] - row['min_ts'])\
    ['delta'].mean()

Timedelta('463 days 21:28:27.449612384')

# Задание 4

Дана статистика услуг перевозок клиентов компании по типам (см. файл “Python_13_join.ipynb” в разделе Материалы для лекции «Продвинутый pandas» ---- Ноутбуки к лекции «Продвинутый pandas»).

Необходимо сформировать две таблицы:
* таблицу с тремя типами выручки для каждого client_id без указания адреса клиента
* аналогичную таблицу по типам выручки с указанием адреса клиента

Обратите внимание, что в процессе объединения таблиц данные не должны теряться.

#### get `rzd`, `auto`, `air`, `client_base`

In [ ]:
rzd = pd.DataFrame(
    {
        'client_id': [111, 112, 113, 114, 115],
        'rzd_revenue': [1093, 2810, 10283, 5774, 981]
    }
)
auto = pd.DataFrame(
    {
        'client_id': [113, 114, 115, 116, 117],
        'auto_revenue': [57483, 83, 912, 4834, 98]
    }
)
air = pd.DataFrame(
    {
        'client_id': [115, 116, 117, 118],
        'air_revenue': [81, 4, 13, 173]
    }
)
client_base = pd.DataFrame(
    {
        'client_id': [111, 112, 113, 114, 115, 116, 117, 118],
        'address': ['Комсомольская 4', 'Энтузиастов 8а', 'Левобережная 1а', 'Мира 14', 'ЗЖБИиДК 1', 
                    'Строителей 18', 'Панфиловская 33', 'Мастеркова 4']
    }
)

#### solution

In [ ]:
revenue = rzd\
    .merge(auto, how='outer', left_on=['client_id'], right_on=['client_id'])\
    .merge(air, how='outer', left_on=['client_id'], right_on=['client_id']).fillna(value=0)

In [ ]:
revenue

,client_id,rzd_revenue,auto_revenue,air_revenue
0,111,1093.0,0.0,0.0
1,112,2810.0,0.0,0.0
2,113,10283.0,57483.0,0.0
3,114,5774.0,83.0,0.0
4,115,981.0,912.0,81.0
5,116,0.0,4834.0,4.0
6,117,0.0,98.0,13.0
7,118,0.0,0.0,173.0


In [ ]:
client_revenue = client_base.merge(revenue, how='outer', left_on=['client_id'], right_on=['client_id']).fillna(value='')

In [ ]:
client_revenue

,client_id,address,rzd_revenue,auto_revenue,air_revenue
0,111,Комсомольская 4,1093.0,0.0,0.0
1,112,Энтузиастов 8а,2810.0,0.0,0.0
2,113,Левобережная 1а,10283.0,57483.0,0.0
3,114,Мира 14,5774.0,83.0,0.0
4,115,ЗЖБИиДК 1,981.0,912.0,81.0
5,116,Строителей 18,0.0,4834.0,4.0
6,117,Панфиловская 33,0.0,98.0,13.0
7,118,Мастеркова 4,0.0,0.0,173.0
